# Phase Identification NN

Developement of a neural network capable of identifying the phase of a 2D system (liquid, hexatic or solid)

In [3]:
#import tensorflow as tf

import glob
import numpy as np
import matplotlib.pyplot as plt

#print(tf.__version__)

## Load data

First, let us load the data

In [36]:
# loop over all the runs
path = 'ordered_128/Pe0'
conf_counter = 0
for fi in glob.glob(path + '/fi*'):
    # extract the density from the path name
    #print(fi)
    density, run = tuple(fi.split('/')[2].split('_'))
    density = float(density.split('fi')[1])
    print(density, run)
    # loop over all the trajectories
    phase_label = 0 # liquid
    if density > 0.72:
        phase_label = 1
    if density > 0.73:
        phase_label = 2
    #print(phase_label)
    
    for trj in glob.glob(fi + '/Trj/xyz.dump.*'):
        # load the trajectory
        #print(phase_label, trj)
        data = np.loadtxt(trj, skiprows=9)
        
        train_data = np.array([data, phase_label])
        if phase_label>1:
            print(train_data)

        conf_counter += 1

print(conf_counter)

0.708 8


/tmp/ipykernel_141825/1859575872.py:23: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_data = np.array([data, phase_label])


KeyboardInterrupt: 

## Data selection


## Set the model

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(512,2)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3)
]